## Importando o arquivo

In [8]:
import pandas as pd

nome_arquivo = r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\coffee.csv"
dados_cafe = pd.read_csv(nome_arquivo)


In [9]:
dados_cafe.head()

,Date,Open,High,Low,Close,Volume,Currency
0,2019-01-02,101.6,102.65,99.35,99.50,22181,USD
1,2019-01-03,99.5,103.25,99.45,102.15,21488,USD
2,2019-01-04,102.3,103.00,100.40,101.60,16115,USD
3,2019-01-07,101.5,103.45,101.15,102.75,15307,USD
4,2019-01-08,102.9,106.05,102.35,105.05,32987,USD


## Lendo as estruturas de cada coluna para entender melhor o CSV

In [10]:
print(dados_cafe.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Currency'], dtype='object')


In [11]:
print(dados_cafe.dtypes)

Date         object
Open        float64
High        float64
Low         float64
Close       float64
Volume        int64
Currency     object
dtype: object


In [12]:
dados_cafe['Date'] = pd.to_datetime(dados_cafe['Date'])

## Aqui fiz um ETL de Datetime pois ele estava como object

In [13]:
print(dados_cafe.dtypes)

Date        datetime64[ns]
Open               float64
High               float64
Low                float64
Close              float64
Volume               int64
Currency            object
dtype: object


## Fiz uma analise para ver se tinha algum Null na coluna e rodei um código em Python para dar um double check.

In [18]:
print(dados_cafe.isnull().sum())

Date        0
Open        0
High        0
Low         0
Close       0
Volume      0
Currency    0
dtype: int64


In [29]:
for data in dados_cafe['Date']:
    try:
        pd.to_datetime(data)
    except ValueError:
        print("Data fora do padrão: ", data)

In [15]:
import sqlite3

nome_arquivo = r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\exchange_rates.db"
conexao = sqlite3.connect(nome_arquivo)

# Para executar uma consulta SQL e obter os resultados em um DataFrame pandas:
consulta_sql = "SELECT * FROM exchange_rates"
dados_taxas_cambio = pd.read_sql_query(consulta_sql, conexao)

# Para fechar a conexão com o banco de dados:
conexao.close()

In [16]:
print(conexao)

In [17]:
conexao = sqlite3.connect(nome_arquivo)

# Para executar uma consulta SQL para obter as informações sobre a tabela:
consulta_sql = "PRAGMA table_info('exchange_rates')"
cursor = conexao.execute(consulta_sql)
colunas = cursor.fetchall()

# Para imprimir na tela as informações sobre a tabela:
print(colunas)

# Para fechar a conexão com o banco de dados:
conexao.close()


[(0, 'id', 'INTEGER', 1, None, 1), (1, 'Date', 'DATE', 0, None, 0), (2, 'Currency', 'VARCHAR(3)', 0, None, 0), (3, 'Rate', 'FLOAT', 0, None, 0)]


## Fazendo as análises do Desafio.

In [32]:
import sqlite3
import pandas as pd

# Leitura do arquivo coffee.csv para um DataFrame pandas
dados_cafe = pd.read_csv(r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\coffee.csv")

# Criação de conexão com o banco de dados em memória
conexao = sqlite3.connect(":memory:")

# Carregamento dos dados do DataFrame para uma tabela "cafe" no banco de dados em memória
dados_cafe.to_sql("cafe", conexao, index=False)

# Consulta SQL para obter o maior volume negociado de café no dia e as cotações de fechamento
consulta_sql = """
SELECT Date, Close, Volume
FROM cafe
WHERE Volume = (SELECT MAX(Volume) FROM cafe)
"""
resultado = pd.read_sql_query(consulta_sql, conexao)
print("Maior volume negociado de café no dia e as cotações de fechamento:")
print(resultado)

# Consulta SQL para obter o total de café negociado por ano e as cotações
consulta_sql = """
SELECT STRFTIME('%Y', Date) AS Ano, SUM(Volume) AS Total_Negociado, AVG(Close) AS Cotacao_Media
FROM cafe
GROUP BY Ano
"""
resultado = pd.read_sql_query(consulta_sql, conexao)
print("Total de café negociado por ano e as cotações:")
print(resultado)

# Consulta SQL para obter a média de volume negociado mensal e anual
consulta_sql = """
SELECT STRFTIME('%Y', Date) AS Ano, STRFTIME('%m', Date) AS Mes, AVG(Volume) AS Media_Volume_Negociado
FROM cafe
GROUP BY Ano, Mes
"""
resultado = pd.read_sql_query(consulta_sql, conexao)
print("Média de volume negociado mensal e anual:")
print(resultado)

# Fechamento da conexão com o banco de dados em memória
conexao.close()

Maior volume negociado de café no dia e as cotações de fechamento:
         Date  Close  Volume
0  2019-04-11  90.25   62045
Total de café negociado por ano e as cotações:
    Ano  Total_Negociado  Cotacao_Media
0  2019          4011645     101.141270
1  2020          3078378     111.017589
2  2021          3127536     168.890278
3  2022          2374254     227.172515
Média de volume negociado mensal e anual:
     Ano Mes  Media_Volume_Negociado
0   2019  01            21827.476190
1   2019  02            17392.684211
2   2019  03             8916.666667
3   2019  04            23584.142857
4   2019  05            13223.409091
5   2019  06            22037.900000
6   2019  07             8819.772727
7   2019  08            15226.227273
8   2019  09             7425.550000
9   2019  10            24074.478261
10  2019  11            21131.950000
11  2019  12             7368.666667
12  2020  01            22863.380952
13  2020  02            17807.947368
14  2020  03             7510.6

## utilizando o request direto da API.

In [53]:
import requests

url = "https://api.apilayer.com/currency_data/historical?date=2019-04-11"

payload = {}
headers= {
  "apikey": "YeAMzndvd2wsvSujKL21yc3UWplNtdV4"
}

response = requests.request("GET", url, headers=headers, data = payload)

status_code = response.status_code
result = response.json()

# Impressão das cotações das moedas Real - BRL, Euro - EUR e Peso Chileno - CLP presentes na resposta
cotacoes = result["quotes"]
print("Cotação do Real - BRL:", cotacoes["USDBRL"])
print("Cotação do Euro - EUR:", cotacoes["USDEUR"])
print("Cotação do Peso Chileno - CLP:", cotacoes["USDCLP"])


Cotação do Real - BRL: 3.857302
Cotação do Euro - EUR: 0.88807
Cotação do Peso Chileno - CLP: 663.898058


# CUIDADO COM ESSA QUERY, ELA SUBSCREVE O ARQUIVO PRONTO CSV DE ANALISE DO DIRETÓRIO.

In [11]:
import sqlite3
import pandas as pd

# Leitura do arquivo coffee.csv para um DataFrame pandas
dados_cafe = pd.read_csv(r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\coffee.csv")

# Criação de conexão com o banco de dados em memória
conexao = sqlite3.connect(":memory:")

# Carregamento dos dados do DataFrame para uma tabela "cafe" no banco de dados em memória
dados_cafe.to_sql("cafe", conexao, index=False)

# Consulta SQL para obter o maior volume negociado de café no dia e as cotações de fechamento
consulta_sql = """
SELECT Date, Close, Volume
FROM cafe
WHERE Volume = (SELECT MAX(Volume) FROM cafe)
"""
resultado = pd.read_sql_query(consulta_sql, conexao)
print("Maior volume negociado de café no dia e as cotações de fechamento:")
print(resultado)
resultado.to_csv(r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\maior_volume_negociado.csv", index=False)

# Consulta SQL para obter o total de café negociado por ano e as cotações
consulta_sql = """
SELECT STRFTIME('%Y', Date) AS Ano, SUM(Volume) AS Total_Negociado, AVG(Close) AS Cotacao_Media
FROM cafe
GROUP BY Ano
"""
resultado = pd.read_sql_query(consulta_sql, conexao)
print("Total de café negociado por ano e as cotações:")
print(resultado)
resultado.to_csv(r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\total_negociado_por_ano.csv", index=False)

# Consulta SQL para obter a média de volume negociado mensal e anual
consulta_sql = """
SELECT STRFTIME('%Y', Date) AS Ano, STRFTIME('%m', Date) AS Mes, AVG(Volume) AS Media_Volume_Negociado
FROM cafe
GROUP BY Ano, Mes
"""
resultado = pd.read_sql_query(consulta_sql, conexao)
print("Média de volume negociado mensal e anual:")
print(resultado)
resultado.to_csv(r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\media_volume_negociado_mensal_anual.csv", index=False)

# Fechamento da conexão com o banco de dados em memória
conexao.close()


Maior volume negociado de café no dia e as cotações de fechamento:
         Date  Close  Volume
0  2019-04-11  90.25   62045
Total de café negociado por ano e as cotações:
    Ano  Total_Negociado  Cotacao_Media
0  2019          4011645     101.141270
1  2020          3078378     111.017589
2  2021          3127536     168.890278
3  2022          2374254     227.172515
Média de volume negociado mensal e anual:
     Ano Mes  Media_Volume_Negociado
0   2019  01            21827.476190
1   2019  02            17392.684211
2   2019  03             8916.666667
3   2019  04            23584.142857
4   2019  05            13223.409091
5   2019  06            22037.900000
6   2019  07             8819.772727
7   2019  08            15226.227273
8   2019  09             7425.550000
9   2019  10            24074.478261
10  2019  11            21131.950000
11  2019  12             7368.666667
12  2020  01            22863.380952
13  2020  02            17807.947368
14  2020  03             7510.6

## Utilizei alguns comandos em python para transformação da tabela Raw - bronze - silver - gold, para ter as cotações, os códigos estão todos junto ao documento git.

In [41]:
import pandas as pd

nome_arquivo2 = r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\coffee_gold.csv"
dados_cafe_gold = pd.read_csv(nome_arquivo2)

In [42]:
dados_cafe_gold.head()

,Date,Open,High,Low,Close,Volume,Currency,USD-BRL,USD-CLP,USD-EUR,Volume_USD-BRL,Volume_USD-CLP,Volume_USD-EUR
0,1/2/2019,101.6,102.65,99.35,99.50,22181,USD,3.789851,696.820523,0.883585,84062.68503,15456176.02,19598.79889
1,1/3/2019,99.5,103.25,99.45,102.15,21488,USD,3.756597,693.799154,0.877920,80721.75634,14908356.22,18864.74496
2,1/4/2019,102.3,103.00,100.40,101.60,16115,USD,3.715204,681.203912,0.877535,59870.51246,10977601.04,14141.47653
3,1/7/2019,101.5,103.45,101.15,102.75,15307,USD,3.735601,681.249605,0.871125,57180.84451,10427887.70,13334.31038
4,1/8/2019,102.9,106.05,102.35,105.05,32987,USD,3.713906,680.400592,0.872925,122510.61720,22444374.33,28795.17698


In [43]:
dados_cafe_gold.dtypes

Date               object
Open              float64
High              float64
Low               float64
Close             float64
Volume              int64
Currency           object
USD-BRL           float64
USD-CLP           float64
USD-EUR           float64
Volume_USD-BRL    float64
Volume_USD-CLP    float64
Volume_USD-EUR    float64
dtype: object

## tudo ok, vamos fazer a analise novamente agora com a planilha de valores e volumes, antes vou transformar o arquivo em Parquet para pesar menos.

In [12]:
import pandas as pd

# Defina o caminho do arquivo de entrada e saída
input_csv = r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\coffee_gold.csv"
output_parquet = r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\coffee_gold.parquet"

# Leia o arquivo CSV como um DataFrame do pandas
df = pd.read_csv(input_csv)

# Salve o DataFrame como um arquivo Parquet
df.to_parquet(output_parquet, engine='pyarrow', index=False)


In [27]:
df.head()

Row(Date='1/2/2019', Open=101.6, High=102.65, Low=99.35, Close=99.5, Volume=22181, Currency='USD', USDBRL=3.789851, USDCLP=696.820523, USDEUR=0.883585, Volume_USDBRL=84062.68503, Volume_USDCLP=15456176.02, Volume_USDEUR=19598.79889)

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month

# Criando uma SparkSession
spark = SparkSession.builder.master("local").appName("CoffeeAnalysis").getOrCreate()


In [25]:
input_parquet = r"C:\Users\andre\Documents\Kaggle\PISMO\Desafio\coffee_gold.parquet"

# Leia o arquivo Parquet como um DataFrame do PySpark (cuidado com o arquivo pois estou lendo o parquet, ele está dentro do git)
df = spark.read.parquet(input_parquet)

In [ ]:
## Perguntas do desafio:

## Utilizando pandas para ter um double check.

In [39]:
import pandas as pd

# Carregar o arquivo CSV
coffee_gold = pd.read_csv("C:/Users/andre/Documents/Kaggle/PISMO/Desafio/coffee_gold.csv")
coffee_gold['Date'] = pd.to_datetime(coffee_gold['Date'])

# Maior volume negociado de café no dia e as cotações de fechamento
max_volume = coffee_gold.loc[coffee_gold['Volume'].idxmax()]
print("Maior volume negociado de café no dia e as cotações de fechamento:")
print(max_volume)

# Total de café negociado por ano e as cotações
coffee_gold['Ano'] = coffee_gold['Date'].dt.year
total_negociado_por_ano = coffee_gold.groupby('Ano').agg({
    'Volume': 'sum',
    'USD-BRL': 'mean',
    'USD-CLP': 'mean',
    'USD-EUR': 'mean'
})
print("Total de café negociado por ano e as cotações:")
print(total_negociado_por_ano)

# Média de volume negociado mensal e anual
coffee_gold['Mes'] = coffee_gold['Date'].dt.month
media_volume_mensal_anual = coffee_gold.groupby(['Ano', 'Mes']).agg({
    'Volume': 'mean',
    'USD-BRL': 'mean',
    'USD-CLP': 'mean',
    'USD-EUR': 'mean'
})
print("Média de volume negociado mensal e anual:")
print(media_volume_mensal_anual)

Maior volume negociado de café no dia e as cotações de fechamento:
Date              2019-04-11 00:00:00
Open                             94.1
High                             94.3
Low                             90.15
Close                           90.25
Volume                          62045
Currency                          USD
USD-BRL                      3.857302
USD-CLP                    663.898058
USD-EUR                       0.88807
Volume_USD-BRL            239326.3026
Volume_USD-CLP            41191555.01
Volume_USD-EUR            55100.30315
Name: 69, dtype: object
Total de café negociado por ano e as cotações:
       Volume   USD-BRL     USD-CLP   USD-EUR
Ano                                          
2019  4011645  3.945154  703.635667  0.893084
2020  3078378  5.164605  792.334426  0.876617
2021  3152066  5.398363  760.687902  0.845883
2022  2401434  5.117718  853.370903  0.933512
Média de volume negociado mensal e anual:
                Volume   USD-BRL     USD-CLP   USD

In [ ]:
## para fazer as consultas vou utilziar o spark, como estou trabalhando com python vou utilizar o fram. Pyspark (atenção para a versão) // Perguntas do desafio:

In [59]:
!pip install pyspark

In [52]:
from pyspark.sql import SparkSession

# Inicialize a SparkSession
spark = SparkSession.builder \
    .appName("Pandas to Spark DataFrame Conversion") \
    .getOrCreate()

# Converter o pandas DataFrame em PySpark DataFrame
coffee_gold_spark = spark.createDataFrame(coffee_gold)

# Adicionar as colunas 'Ano' e 'Mes'
coffee_gold_spark = coffee_gold_spark.withColumn('Ano', year(coffee_gold_spark['Date']))
coffee_gold_spark = coffee_gold_spark.withColumn('Mes', month(coffee_gold_spark['Date']))

# Crie uma visualização temporária para executar consultas SQL
coffee_gold_spark.createOrReplaceTempView("coffee_gold")


## Pyspark com python 3.11

In [68]:
import os

os.environ["PYSPARK_PYTHON"] = "C:\\Users\\andre\\AppData\\Local\\Programs\\Python\\Python311\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "C:\\Users\\andre\\AppData\\Local\\Programs\\Python\\Python311\\python.exe"
!pip install pyspark

In [ ]:
total_cafe_ano = spark.sql("""
SELECT Ano,
       SUM(Volume) as Total_Negociado,
       AVG(Close) as Cotacao_Media,
       AVG(`USD-BRL`) as Media_USDBRL,
       AVG(`USD-CLP`) as Media_USDCLP,
       AVG(`USD-EUR`) as Media_USDEUR
FROM coffee_gold
GROUP BY Ano
""")

total_cafe_ano.show()

In [ ]:
maior_volume_dia = spark.sql("""
SELECT Date, Close, Volume, `USD-BRL`, `USD-CLP`, `USD-EUR`
FROM coffee_gold
WHERE Volume = (SELECT MAX(Volume) FROM coffee_gold)
""")

maior_volume_dia.show()

In [ ]:
media_volume_mensal_anual = spark.sql("""
SELECT Ano, Mes,
       AVG(Volume) as Media_Volume_Negociado,
       AVG(`USD-BRL`) as Media_USDBRL,
       AVG(`USD-CLP`) as Media_USDCLP,
       AVG(`USD-EUR`) as Media_USDEUR
FROM coffee_gold
GROUP BY Ano, Mes
ORDER BY Ano, Mes
""")

media_volume_mensal_anual.show()

In [74]:
## disperção para analisar os quartis.

In [73]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Estatísticas descritivas básicas
stats = coffee_gold.describe()
print(stats)

# 2. Preços médios por ano e por mês
coffee_gold['Ano'] = pd.DatetimeIndex(coffee_gold['Date']).year
coffee_gold['Mes'] = pd.DatetimeIndex(coffee_gold['Date']).month

average_prices = coffee_gold.groupby(['Ano', 'Mes']).mean()
print(average_prices)



             Open        High         Low       Close        Volume  \
count  926.000000  926.000000  926.000000  926.000000    926.000000   
mean   145.303790  147.227851  143.375065  145.178467  13653.912527   
std     51.640715   52.379308   50.894639   51.804886  12702.120368   
min     87.050000   87.600000   86.350000   86.650000      0.000000   
25%    103.700000  104.925000  102.350000  103.562500     28.250000   
50%    122.500000  124.250000  120.575000  122.500000  14666.000000   
75%    199.837500  203.312500  194.437500  199.050000  22584.750000   
max    259.100000  260.450000  253.650000  258.350000  62045.000000   

          USD-BRL      USD-CLP     USD-EUR  Volume_USD-BRL  Volume_USD-CLP  \
count  926.000000   926.000000  926.000000      926.000000    9.260000e+02   
mean     4.888106   770.623271    0.883023    65391.244441    1.042613e+07   
std      0.658788    67.806046    0.040162    59457.432871    9.539160e+06   
min      3.644305   648.103082    0.810403      